# Running ensembles of SUMMA simulations

`pysumma` offers an `Ensemble` class which is useful for running multiple simulations with varying options. These options can be parameter values, model structures, different locations made up of different file managers, or combinations of any of these. To demonstrate the `Ensemble` capabilities we will step through each individually. As usual we will begin with some imports and definition of some global variables.

In [ ]:
import numpy as np
import pysumma as ps
import matplotlib.pyplot as plt
from pprint import pprint

file_manager = 'PATH/TO/FILE_MANAGER'
summa_exe = 'summa.exe'

## Changing decisions

The `Ensemble` object mainly operates by taking configuration dictionaries. These can be defined manually, or can be defined through the use of helper functions which will be described later. For now, we will look at how to run models using different options for the `stomResist` and `soilStress` decisions. This is done by providing a dictionary of these mappings inside of a `decisions` key of the overall configuration. The `decisions` key is one of several special configuration keys which `pysumma` knows how to manipulate under the hood. We will explore the others later.

This configuration is used to construct the `Ensemble` object, which also takes the SUMMA executable, a file manager path, and optionally the `num_workers` argument. The `num_workers` argument is used to automatically run these ensemble members in parallel. Here we define it as 2, since that's how many different runs we will be completing. You _can_ set this to a higher number than your computer has CPU cores, but you won't likely see any additional speedup by doing so.

We then run the ensemble through the `run` method which works similarly to the `Simulation` object. After running the ensemble we check the status by running the `summary` method. This will return a dictionary outlining any successes or failures of each of the members. In the event of any failures you can later inspect the underlying `Simulation` objects that are internal to the `Ensemble`. We will demonstrate how to do this later in the tutorial.

In [ ]:
config = {
    'run0': {'decisions':{'stomResist': 'Jarvis',
                          'soilStress': 'NoahType'}},
    'run1': {'decisions':{'stomResist': 'BallBerry',
                          'soilStress': 'CLM_Type'}},
}

decision_ens = ps.Ensemble(summa_exe, config, file_manager, num_workers=2)
decision_ens.run('local')
print(decisions_ens.summary())

<br>
Note in the previous example we didn't run every combination of `stomResist` and `soilStress` that we could have. When running multiple configurations it often becomes unwieldy to type out the full configuration that you are attempting to run, so some helper functions have been implemented to make this a bit easier. In the following cell we demonstrate this. You can see that the new output will show we have 4 configurations to run, each with a unique set of decisions. The names are just the decisions being set, delimited by `++` so that it is easy to keep track of each run.

In [ ]:
decisions_to_run = {
    'stomResist': ['Jarvis', 'BallBerry'],
    'soilStress': ['NoahType', 'CLM_Type']
}

config = ps.ensemble.decision_product(decisions_to_run)
pprint(config)

<br>

## Changing parameters

Similarly, you can change parameter values to do sensitivity experiments:

In [ ]:
config = {
    'run0': {'trial_params': {'windReductionParam': 0.1,
                              'canopyWettingFactor': 0.1}}
    'run1': {'trial_params': {'windReductionParam': 0.9,
                              'canopyWettingFactor': 0.9}}
}

param_ens = ps.Ensemble(summa_exe, config, file_manager, num_workers=2)
param_ens.run('local')
print(param_ens.summary())

<br> 

As you can tell, it can quickly become tiresome to type out every parameter value you want to set. To that end we also have a helper function for setting up these parameter sensitivity experiments. Now you can see that we will end up with 4 configurations, as before. We won't run this because it may take longer than is instructional. But, you can modify this notebook if you wish to see the effect each of these have.

In [ ]:
parameters_to_run = {
    'windReductionParam': [0.1, 0.9],
    'canopyWettingFactor': [0.1, 0.9]
}

config = ps.ensemble.trial_parameter_product(parameters_to_run)
pprint(config)


## Running multiple sites via file managers

As you may have guessed, you can also define an `Ensemble` by providing a list of file managers. This is useful for running multiple sites which can't be collected into a multi HRU run because they have different simulation times, are disjointly located, or for any other reason. It is important to note that in this case we don't provide the `Ensemble` constructor a `file_manager` argument, as it is now provided in the configuration.

In [ ]:
config = {
    'run0': {'file_manager': 'PATH/TO/FM0'},
    'run1': {'file_manager': 'PATH/TO/FM1'},
}

manager_ens = ps.Ensemble(summa_exe, config, num_workers=2)
manager_ens.run('local')
print(manager_ens.summary())

In [ ]:
managers_to_run = {
    'file_manager': ['PATH/TO/FM0', 'PATH/TO/FM1']
}

config = ps.ensemble.file_manager_product(managers_to_run)
pprint(config)


## Combining ensemble types

Each of these abilities are useful in their own right, but the ability to combine them into greater ensembles provides a very flexible way to explore multiple hypotheses. To this end we also provide a helper function which can facilitate running these larger experiments. We won't print out the entire configuration here, since it's quite long. Instead we show that this would result in 32 SUMMA simulations. For that reason we also won't run this experment by default, though you can if you wish to.

In [ ]:
config = ps.ensemble.total_product(dec_conf=decisions_to_run, 
                                   param_trial_conf=parameters_to_run, 
                                   fman_conf=managers_to_run)
print(len(config))

For illustrative purpose we show the first key of this configuration

In [ ]:
print(list(config.keys())[0])

<br>

As you can see, the keys (or names of the runs) grows when you include more options. This can be a problem for some operating systems/filesystems, along with being very hard to read. So, we also have a flag here that makes things more compact

In [ ]:
config = ps.ensemble.total_product(dec_conf=decisions_to_run, 
                                   param_trial_conf=parameters_to_run, 
                                   fman_conf=managers_to_run,
                                   sequential_keys=True
                                  )
print(list(config.keys())[0])


## Interacting with the underlying `Simulation` objects

After all of the configuration madness it is probably worth figuring out how to actually deal with the simulations that get run via the `Ensemble`. To get back on track we'll just run the first decision ensemble again. Once it's been run and the summary reports success we can dig in deeper.

In [ ]:
config = {
    'run0': {'decisions':{'stomResist': 'Jarvis',
                          'soilStress': 'NoahType'}},
    'run1': {'decisions':{'stomResist': 'BallBerry',
                          'soilStress': 'CLM_Type'}},
}

decision_ens = ps.Ensemble(summa_exe, config, file_manager, num_workers=2)
decision_ens.run('local')
print(decisions_ens.summary())

<br>

To access the simulations individually you can use the `.simulations` attribute, which is simply a dictionary of the `Simulation` objects mapped by the `name` that's used to run the simulation. Let's loop through and print the decisions directly from the `Simulation` as proof things worked.

In [ ]:
for n, sim in decision_ens.items():
    print(f'{n} used {sim.decisions["stomResist"]} for the stomResist option and {sim.decisions["soilStress"]} for soilStress')

We also provide some functionality to make it easier to wrangle the data together via the `merge_output` method.

In [ ]:
ens_ds = decision_ens.merge_output()
ens_ds['scalarLatHeatTotal'].plot.line(x='time')